In [1]:
%load_ext autoreload
%autoreload 2
import os
import shutil
import torch
import numpy as np
import fairseq
import argparse

In [2]:
fairseq.__version__

'0.12.2'

In [3]:
path = "/mnt/dl/"

In [4]:
src_lang, tgt_lang = 'en', 'fr'

In [5]:
datapath = os.path.join(path, 'Translation', 'WMT_15', src_lang + '-' + tgt_lang)

In [6]:
os.listdir(datapath)

['test.tok.clean.dl.bpe.40000.en',
 'valid.tok.clean.dl.bpe.40000.en',
 'valid.tok.clean.dl.en',
 'valid.tok.fr',
 'test.tok.clean.dl.en',
 'test.tok.clean.en',
 'test.fr',
 'test.tok.clean.fr',
 'valid.tok.clean.dl.fr',
 'train.tok.clean.dl.en',
 'train.tok.clean.dl.en-fr',
 'vocab.bpe.40000',
 'train.tok.clean.fr',
 'test.tok.clean.dl.fr',
 'train.fr',
 'test.tok.clean.dl.bpe.40000.fr',
 'test.tok.fr',
 'valid.tok.en',
 'train.tok.en',
 'train.tok.clean.dl.fr',
 'valid.fr',
 'train.tok.fr',
 'bpe.40000',
 'valid.tok.clean.en',
 'train.en',
 'mosesdecoder',
 'test.en',
 'valid.tok.clean.dl.bpe.40000.fr',
 'train.tok.clean.en',
 'test.tok.en',
 'valid.en',
 'train.tok.clean.dl.bpe.40000.fr',
 'valid.tok.clean.fr',
 'train.tok.clean.dl.bpe.40000.en',
 'train.tok.clean.dl.bpe.40000.en-fr']

In [7]:
new_path = os.path.join(path, 'fairseq', 'Translation')
os.makedirs(new_path, exist_ok=True)

In [8]:
def bump_new_files(path, new_path, tag, size=1000):
    for lang in [src_lang, tgt_lang]:
        with open(os.path.join(path, f'{tag}.tok.clean.dl.bpe.40000.{lang}')) as f, \
           open(os.path.join(new_path, f'{tag}.tok.clean.dl.bpe.40000.{lang}'), 'w') as sf:
            lines = []
            for i, line in enumerate(f):
                if i == size:
                    break
                lines.append(line)
            
            sf.writelines(lines)


In [9]:
bump_new_files(datapath, new_path, 'train', 10000)

In [10]:
with open(os.path.join(new_path, f'train.tok.clean.dl.bpe.40000.{src_lang}')) as f:
    for i, line in enumerate(f):
        if i == 10:
            break
        print(line)

In his briefing on economic development , Al Hor@@ ner will give you details of programs we fund to foster partnerships between the private sector and First Nations and Inuit communities , in areas like resource development projects , for example .

( b ) Positive aspects

Activities of the second type will be a major determin@@ ant of the successful implementation of a language policy .

Emergency suppl@@ emental requirements with respect to the support account for peacekeeping operations would amount to $ 11,@@ 36@@ 5,@@ 400 , comprising $ 5,@@ 00@@ 6,@@ 900 for the establishment of 181 new posts ( 2 D-2 , 1 D-1 , 8 P-5 , 104 P-4 / 3 , 8 P-2 / 1 , 8 GS ( Principal level ( PL ) and 50 GS ( OL ) ) and $ 6,@@ 35@@ 8,@@ 500 for non-@@ post requirements .

Order No. 1991-@@ R-@@ 140 dated May 17 , 1991 - City of Lethbridge , mileage 1@@ 10.@@ 95 Tab@@ er Subdivision .

&quot; Judge Gold@@ stone has confirmed that there were war crimes committed by the Israeli forces and also crimes agains

In [11]:
with open(os.path.join(new_path, f'train.tok.clean.dl.bpe.40000.{tgt_lang}')) as f:
    for i, line in enumerate(f):
        if i == 10:
            break
        print(line)

Dans sa présentation sur le développement économique , M. Al Hor@@ ner vous donnera des détails sur les programmes que nous finan@@ çons pour favoriser l&apos; établissement de partenariats entre le secteur privé et les collectivités des Premières nations et inu@@ ites dans des domaines comme celui de l&apos; exploitation des ressources naturelles .

b ) Aspects positifs

Pour la mise en oeuvre , la deuxième catégorie d&apos; activités constitue un déterminant essentiel du succès de la politique linguistique .

Les ressources supplémentaires à pré@@ lever d&apos; urgence sur le compte d&apos; appui aux opérations de maintien de la paix se mon@@ teraient à 11 365 400 dollars , soit 5 006 900 dollars pour la création de 181 postes &#91; 2 D-2 , 1 D-1 , 8 P-5 , 104 P-4 / 3 , 8 P-2 / 1 , 8 GS ( 1re classe ) et 50 GS ( autres classes ) &#93; et 6 358 500 dollars pour les objets de dépense autres que les postes .

L&apos; arrêté no 1991-@@ R-@@ 140 du 17 mai 1991 - ville de Lethbridge , poin

In [12]:
shutil.copy2(os.path.join(datapath,  'bpe.40000'), os.path.join(new_path,  'bpe.40000'))

'/mnt/dl/fairseq/Translation/bpe.40000'

In [13]:
destdir = os.path.join(new_path, 'data-bin')
os.makedirs(destdir, exist_ok=True)
destdir

'/mnt/dl/fairseq/Translation/data-bin'

In [14]:
parser = argparse.ArgumentParser()

In [15]:
parser.add_argument('--source-lang', default=src_lang)
parser.add_argument('--target-lang', default=tgt_lang)
parser.add_argument('--trainpref', default=os.path.join(new_path, f'train.tok.clean.dl.bpe.40000'))
parser.add_argument('--destdir', default=destdir)

_StoreAction(option_strings=['--destdir'], dest='destdir', nargs=None, const=None, default='/mnt/dl/fairseq/Translation/data-bin', type=None, choices=None, help=None, metavar=None)

In [16]:
args = parser.parse_args([])

In [17]:
args.source_lang

'en'

In [18]:
from fairseq_cli import preprocess
from fairseq import options
from fairseq.data import Dictionary, MMapIndexedDataset, IndexedDataset, IndexedRawTextDataset, PrependTokenDataset, ConcatDataset, StripTokenDataset
from fairseq.binarizer import VocabularyDatasetBinarizer, FileBinarizer
from fairseq.data import data_utils

In [19]:
parser = options.get_preprocessing_parser()

In [20]:
args = parser.parse_args([])

In [21]:
args.__dict__

{'no_progress_bar': False,
 'log_interval': 100,
 'log_format': None,
 'log_file': None,
 'aim_repo': None,
 'aim_run_hash': None,
 'tensorboard_logdir': None,
 'wandb_project': None,
 'azureml_logging': False,
 'seed': 1,
 'cpu': False,
 'tpu': False,
 'bf16': False,
 'memory_efficient_bf16': False,
 'fp16': False,
 'memory_efficient_fp16': False,
 'fp16_no_flatten_grads': False,
 'fp16_init_scale': 128,
 'fp16_scale_window': None,
 'fp16_scale_tolerance': 0.0,
 'on_cpu_convert_precision': False,
 'min_loss_scale': 0.0001,
 'threshold_loss_scale': None,
 'amp': False,
 'amp_batch_retries': 2,
 'amp_init_scale': 128,
 'amp_scale_window': None,
 'user_dir': None,
 'empty_cache_freq': 0,
 'all_gather_list_size': 16384,
 'model_parallel_size': 1,
 'quantization_config_path': None,
 'profile': False,
 'reset_logging': False,
 'suppress_crashes': False,
 'use_plasma_view': False,
 'plasma_path': '/tmp/plasma',
 'criterion': 'cross_entropy',
 'tokenizer': None,
 'bpe': None,
 'optimizer': No

In [22]:
args.source_lang = src_lang
args.target_lang = tgt_lang
args.trainpref = os.path.join(new_path, f'train.tok.clean.dl.bpe.40000')
args.joined_dictionary = True
args.nwordssrc = 2048 # Dict Size 
args.nwordstgt = 2048 # Dict Size 
args.destdir = destdir

In [23]:
preprocess.main(args)

2023-06-17 15:52:12 | INFO | fairseq_cli.preprocess | Namespace(no_progress_bar=False, log_interval=100, log_format=None, log_file=None, aim_repo=None, aim_run_hash=None, tensorboard_logdir=None, wandb_project=None, azureml_logging=False, seed=1, cpu=False, tpu=False, bf16=False, memory_efficient_bf16=False, fp16=False, memory_efficient_fp16=False, fp16_no_flatten_grads=False, fp16_init_scale=128, fp16_scale_window=None, fp16_scale_tolerance=0.0, on_cpu_convert_precision=False, min_loss_scale=0.0001, threshold_loss_scale=None, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, user_dir=None, empty_cache_freq=0, all_gather_list_size=16384, model_parallel_size=1, quantization_config_path=None, profile=False, reset_logging=False, suppress_crashes=False, use_plasma_view=False, plasma_path='/tmp/plasma', criterion='cross_entropy', tokenizer=None, bpe=None, optimizer=None, lr_scheduler='fixed', scoring='bleu', task='translation', source_lang='en', target_lang='fr', tr

In [24]:
args.task

'translation'

In [25]:
src_dict = Dictionary.load(os.path.join(destdir, 'dict.en.txt'))

In [26]:
src_dict.symbols[:10]

['<s>', '<pad>', '</s>', '<unk>', ',', '.', 'de', 'the', 'of', 'la']

In [27]:
os.listdir(destdir)

['train.en-fr.fr.idx',
 'train.en-fr.fr.bin',
 'dict.fr.txt',
 'preprocess.log',
 'dict.en.txt',
 'train.en-fr.en.idx',
 'train.en-fr.en.bin']

In [28]:
idx_ds = MMapIndexedDataset(os.path.join(destdir, "train.en-fr.en"))

In [29]:
idx_ds.sizes

array([43,  6, 21, ..., 41, 26, 26], dtype=int32)

In [30]:
idx_ds.sizes.sum()

301498

In [31]:
lines = []
with open(os.path.join(destdir, "train.en-fr.en.bin"), 'rb') as f:
    while True:
        data = f.read(1024)
        lines.extend(np.frombuffer(data, dtype=np.uint16).tolist())
        if not data: break
    
    data = []
    pos = 0
    for size in idx_ds.sizes:
        data.append(lines[pos: pos + size])
        pos += size
    
    
        

In [32]:
[len(line) for line in data] == idx_ds.sizes.tolist()

True

In [33]:
type(idx_ds._bin_buffer_mmap)

numpy.memmap

In [34]:
idx_ds._index.dtype

numpy.uint16

In [35]:
np.frombuffer(idx_ds._bin_buffer, dtype=idx_ds._index.dtype, count=idx_ds.sizes[0])

array([  91,  382,    3,   27,  482,  147,    4,    3,    3,    3,   76,
       1601,  156,    3,    8, 1155,  125,    3,   13,    3,    3,  205,
          7, 1400,  981,   10, 1002,   82,   10,    3, 1407,    4,   20,
        488,  592,    3,  147,  918,    4,   26, 1010,    5,    2],
      dtype=uint16)

In [36]:
np.array(data[0])

array([  91,  382,    3,   27,  482,  147,    4,    3,    3,    3,   76,
       1601,  156,    3,    8, 1155,  125,    3,   13,    3,    3,  205,
          7, 1400,  981,   10, 1002,   82,   10,    3, 1407,    4,   20,
        488,  592,    3,  147,  918,    4,   26, 1010,    5,    2])

In [37]:
idx_ds._index[1]

(86, 6)

In [38]:
ptr, size  = idx_ds._index[1]

In [39]:
np.frombuffer(idx_ds._bin_buffer, dtype=idx_ds._index.dtype, count=size, 
              offset=ptr)

array([ 21, 219,  19,   3, 807,   2], dtype=uint16)

In [40]:
np.array(data[1])

array([ 21, 219,  19,   3, 807,   2])

In [41]:
src_dataset = data_utils.load_indexed_dataset(os.path.join(destdir,  "train.en-fr.en"), 
                                              dictionary=src_dict,
                                              dataset_impl="mmap")

2023-06-17 15:52:17 | INFO | fairseq.data.data_utils | loaded 10,000 examples from: /mnt/dl/fairseq/Translation/data-bin/train.en-fr.en


In [42]:
src_dataset.__len__()

10000

In [43]:
src_dataset[0]

tensor([  91,  382,    3,   27,  482,  147,    4,    3,    3,    3,   76, 1601,
         156,    3,    8, 1155,  125,    3,   13,    3,    3,  205,    7, 1400,
         981,   10, 1002,   82,   10,    3, 1407,    4,   20,  488,  592,    3,
         147,  918,    4,   26, 1010,    5,    2])

In [44]:
src_dict.bos_index

0

In [45]:
src_dict.eos_index

2

In [46]:
src_dict.string(src_dataset[0].unsqueeze(0))

'In his <unk> on economic development , <unk> <unk> <unk> will give you <unk> of programs we <unk> to <unk> <unk> between the private sector and First Nations and <unk> communities , in areas like <unk> development projects , for example .'

In [47]:
bos_src_dataset = PrependTokenDataset(src_dataset, src_dict.bos())

In [48]:
bos_src_dataset[0]

tensor([   0,   91,  382,    3,   27,  482,  147,    4,    3,    3,    3,   76,
        1601,  156,    3,    8, 1155,  125,    3,   13,    3,    3,  205,    7,
        1400,  981,   10, 1002,   82,   10,    3, 1407,    4,   20,  488,  592,
           3,  147,  918,    4,   26, 1010,    5,    2])

In [49]:
src_dict.string(bos_src_dataset[0].unsqueeze(0))

'In his <unk> on economic development , <unk> <unk> <unk> will give you <unk> of programs we <unk> to <unk> <unk> between the private sector and First Nations and <unk> communities , in areas like <unk> development projects , for example .'

In [50]:
src_dict.index("[en]")

3